Training Data creation from IFC to Labeled point clouds

In [1]:
import ifcopenshell.util
import ifcopenshell.geom as geom
from ifcopenshell.util.selector import Selector
import open3d as o3d
import numpy as np
import multiprocessing
import math
import os

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


Setting of general parameter

In [2]:
voxel_size_ref = 0.01 # voxel size for the downsampling
voxel_size_target = 0.01
init_radius = 0.15 # initial neirest neighbour search radius
c2c_treshold = 0.15 # All points with a c2c-distance larger than this treshold will be labeled as clutter
search_treshold = c2c_treshold # Max distance for normal comparison 

settings = geom.settings()
settings.set(settings.USE_WORLD_COORDS, True)

Setting of the relevant file locations

In [3]:
ifc_file = ifcopenshell.open(r"D:\SAM-Temp\Projects\2021-06-BIM-Gent Loesbergkaai\MODELS\IFC\01203-40-BIM-Gent Loesbergkaai_3D Model IFC_R01.ifc")
pcd = o3d.io.read_point_cloud(r"D:\SAM-Temp\Projects\2021-06-BIM-Gent Loesbergkaai\DATA\PCD\pointcloud.pcd")

WALLS processing

In [4]:
#Select all wall objects in the IFC file
selector = Selector()
walls = selector.parse(ifc_file, '.IfcWall') # This is equivalent to ifc.by_type('IfcWall')

In [ ]:
#Extract the mesh information from the IFC file
meshinfo = []
print(meshinfo)
for ifc_entity in ifc_file.by_type("IfcWall"):
    # print(ifc_entity)
    try: 
        shape = geom.create_shape(settings, ifc_entity)
        ios_vertices = shape.geometry.verts
        ios_faces = shape.geometry.faces

        grouped_verts = [[ios_vertices[i], ios_vertices[i + 1], ios_vertices[i + 2]] for i in range(0, len(ios_vertices), 3)]
        grouped_faces = [[ios_faces[i], ios_faces[i + 1], ios_faces[i + 2]] for i in range(0, len(ios_faces), 3)]

        meshinfo.append([grouped_verts, grouped_faces])
    except:
        print("FAILED: shape creation") 

Error: Session cannot generate requests

In [ ]:
#Create o3d mesh objects from the mesh info extracted from the BIM model.
meshes = []
for geometry in meshinfo:
    vertices = o3d.utility.Vector3dVector(np.asarray(geometry[0]))
    triangles = o3d.utility.Vector3iVector(np.asarray(geometry[1]))

    mesh = o3d.geometry.TriangleMesh(vertices,triangles)

    meshes.append(mesh)


Error: Session cannot generate requests

In [ ]:
# o3d.visualization.draw_geometries(meshes)

In [ ]:
pcd_walls = o3d.geometry.PointCloud()

for submesh in meshes:
    mesh_points = round(submesh.get_surface_area() * 1000)
    submeshpcd = submesh.sample_points_uniformly(number_of_points = mesh_points, use_triangle_normal=True)
    pcd_walls.__iadd__(submeshpcd)

meshvox_walls = pcd_walls.voxel_down_sample(voxel_size_ref)


In [ ]:
# Merge pointclouds from the meshes to 1 pointcloud and save labeles to a separate array
# FAST method

labels =[]
label_id = [ "Ceiling", "Floor", "Wall", "Beam", "Column", "Clutter"]

length_wall = len(meshvox_walls.points)
i = 0
while i < length_wall:
    labels.append("2")
    i =  i+1

meshpcd = o3d.geometry.PointCloud()
meshpcd.__iadd__(meshvox_walls)
print(meshpcd)

In [ ]:
#downsampling of the real point cloud
pcd = pcd.voxel_down_sample(voxel_size_target)
#compute normals for the real pointcloud 
#TODO: Only when the real pointcloud does not contain normals
pcd.estimate_normals()

#compute the cloud to cloud distances between both downsmpled point clouds
c2c = pcd.compute_point_cloud_distance(meshpcd)

In [ ]:
# Filter pointcloud on c2c distances
# FAST method

i=0
percentage = 0
Inlier_1_indeces = []
Outlier_indeces = []

while i < len(c2c):
    if c2c[i] <= c2c_treshold:
        np.asarray(pcd.colors)[i] = [0,1,0]
        Inlier_1_indeces.append(i)
    elif c2c[i] > c2c_treshold:
        np.asarray(pcd.colors)[i] = [1,0,0]
        Outlier_indeces.append(i)

    progress = i/len(c2c)*100
    
    if progress >= percentage:
        print("progress: %s %%" %(round(progress)))
        percentage = percentage + 5
    i = i+1

Final_inlier_pcd = pcd.select_by_index(Inlier_1_indeces)


In [ ]:
# o3d.visualization.draw_geometries([meshpcd])

In [ ]:
# Filter pointcloud on normals and assign labels.


i=0
percentage = 0
Inlier_2_indeces = []
Final_inlier_labels = []
Final_LOAs = []

print("creating a KD-tree")
meshpcd_tree = o3d.geometry.KDTreeFlann(meshpcd)#Create KD tree index

print("Finished creating a KD-tree")
progress = 0
count = 0
radius = init_radius

for i in Inlier_1_indeces:
    [k, idx, d] = meshpcd_tree.search_radius_vector_3d(pcd.points[i], init_radius) #Neighbour Search radius 10cm 
    # print("Selected points in the neighbourhood")
    # meshpcd_sample_points = np.asarray(meshpcd.points)[idx[:], :]
    # meshpcd_sample_normals = np.asarray(meshpcd.normals)[idx[:], :]
    Not_found = True
    i1=0
    while Not_found and i1 < len(idx) and len(idx) > 0:
        if np.abs(np.dot(np.asarray(pcd.normals[i]), np.asarray(meshpcd.normals)[idx[i1],:])) > 0.9 or np.abs(d[i1]) < c2c_treshold/5:
            Not_found = False
            Inlier_2_indeces.append(i)
            Final_inlier_labels.append(labels[idx[i1]])
            Final_LOAs.append(d[i1]) 
        i1 = i1+1

    if Not_found:
        Outlier_indeces.append(i)

    count = count + 1
    progress = count/len(Inlier_1_indeces)*100
    
    if progress >= percentage:
        print("progress: %s %%" %(round(progress)))
        percentage = percentage + 5


print("extracting relevant points")

Final_inlier_pcd = pcd.select_by_index(Inlier_2_indeces)
print(Final_inlier_pcd)
Clutter_pcd = pcd.select_by_index(Outlier_indeces)
print(Clutter_pcd)

In [ ]:
i=0
percentage = 0

Class_0_indeces = []
Class_0_points = 0
Class_0_LOA30 = 0
Class_0_LOA20 = 0
Class_0_LOA10 = 0

Class_1_indeces = []
Class_1_points = 0
Class_1_LOA30 = 0
Class_1_LOA20 = 0
Class_1_LOA10 = 0

Class_2_indeces = []
Class_2_points = 0
Class_2_LOA30 = 0
Class_2_LOA20 = 0
Class_2_LOA10 = 0

Class_3_indeces = []
Class_3_points = 0
Class_3_LOA30 = 0
Class_3_LOA20 = 0
Class_3_LOA10 = 0

Class_4_indeces = []
Class_4_points = 0
Class_4_LOA30 = 0
Class_4_LOA20 = 0
Class_4_LOA10 = 0

Class_clutter_points = len(Clutter_pcd.points)
Clutter_pcd.paint_uniform_color([0.5,0.5,0.5]) #Grey
Final_inlier_pcd.paint_uniform_color([0.5,0.5,0.5]) #Grey

while i < len(np.asarray(Final_inlier_pcd.points)):
    if Final_inlier_labels[i] == "0":
        np.asarray(Final_inlier_pcd.colors)[i] = [0,0,0]
        Class_0_indeces.append(i)
        Class_0_points = Class_0_points + 1
        dist = Final_LOAs[i]
        if dist <= 0.015:
            Class_0_LOA30 = Class_0_LOA30 + 1
        elif dist > 0.015 and dist <=0.05:
            Class_0_LOA20 = Class_0_LOA20 + 1
        elif dist > 0.05:
            Class_0_LOA10 = Class_0_LOA10 + 1

    elif Final_inlier_labels[i] == "1":
        np.asarray(Final_inlier_pcd.colors)[i] = [1,0,0]
        Class_1_indeces.append(i)
        Class_1_points = Class_1_points + 1
        dist = Final_LOAs[i]
        if dist <= 0.015:
            Class_1_LOA30 = Class_1_LOA30 + 1
        elif dist > 0.015 and dist <=0.05:
            Class_1_LOA20 = Class_1_LOA20 + 1
        elif dist > 0.05:
            Class_1_LOA10 = Class_1_LOA10 + 1

    elif Final_inlier_labels[i] == "2":
        np.asarray(Final_inlier_pcd.colors)[i] = [0,1,0]
        Class_2_indeces.append(i)
        Class_2_points = Class_2_points + 1
        dist = Final_LOAs[i]
        if dist <= 0.015:
            Class_2_LOA30 = Class_2_LOA30 + 1
        elif dist > 0.015 and dist <=0.05:
            Class_2_LOA20 = Class_2_LOA20 + 1
        elif dist > 0.05:
            Class_2_LOA10 = Class_2_LOA10 + 1

    elif Final_inlier_labels[i] == "3":
        np.asarray(Final_inlier_pcd.colors)[i] = [0,0,1]
        Class_3_indeces.append(i)
        Class_3_points = Class_3_points + 1
        dist = Final_LOAs[i]
        if dist <= 0.015:
            Class_3_LOA30 = Class_3_LOA30 + 1
        elif dist > 0.015 and dist <=0.05:
            Class_3_LOA20 = Class_3_LOA20 + 1
        elif dist > 0.05:
            Class_3_LOA10 = Class_3_LOA10 + 1

    elif Final_inlier_labels[i] == "4":
        np.asarray(Final_inlier_pcd.colors)[i] = [1,1,0]
        Class_4_indeces.append(i)
        Class_4_points = Class_4_points + 1
        dist = Final_LOAs[i]
        if dist <= 0.015:
            Class_4_LOA30 = Class_4_LOA30 + 1
        elif dist > 0.015 and dist <=0.05:
            Class_4_LOA20 = Class_4_LOA20 + 1
        elif dist > 0.05:
            Class_4_LOA10 = Class_4_LOA10 + 1

    progress = i/len(np.asarray(Final_inlier_pcd.points))*100
    if progress >= percentage:
        print("progress: %s %%" %(round(progress)))
        percentage = percentage + 5
    i= i+1

print("...DONE...")

In [ ]:
print("Class 2:  %s " % label_id[2])
Class_2_pcd = Final_inlier_pcd.select_by_index(Class_2_indeces)
print(Class_2_pcd)
Class_2_filename = "Cloud_" + str(voxel_size_target) + "_" + label_id[2] + ".pcd"
Class_2_location = r"C:\Repo\SCAN2BIM-python\Samples\Test-Loesberg" +"\\" + Class_2_filename
o3d.io.write_point_cloud(Class_2_location, Class_2_pcd, print_progress=True)
print("Saved to:  %s " % Class_2_location)